In [40]:
import numpy as np
import pandas as pd

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from mmm_eval.data import generate_data
from mmm_eval.configs import PyMCConfig
from pymc_marketing.prior import Prior

# Generate data

In [41]:
data = generate_data()
data.to_csv("data.csv", index=False)

X = data.drop(columns=["revenue","quantity"])
y = data["quantity"]

# Fit a PyMC Model

In [ ]:
base_model = MMM(
    date_column="date_week" ,
    channel_columns=["channel_1","channel_2"],
    adstock=GeometricAdstock(l_max=4),
    saturation=LogisticSaturation()
)

base_model.fit(X=X, y=y, chains=4, target_accept=0.85)

# Create a config

To run the evaluation suite, we need to store the configuration used to create the original model.

To do this, we create a `PyMCConfig` object

In [43]:
# We dont need X and y - we'll get those from the input data!
fit_kwargs = { 
    "chains": 4,
    "target_accept": 0.85,
}

base_config = PyMCConfig(base_model, fit_kwargs=fit_kwargs, target_column="quantity")

# Check that we can use the config to fit a PyMC MMM model

In [ ]:
m1 = MMM(**base_config.model_config.config)
m1.fit(X=X, y=y,**base_config.fit_config.config)

# Test a more complex config

In [ ]:
total_spend_per_channel = data[["channel_1", "channel_2"]].sum(axis=0)

spend_share = total_spend_per_channel / total_spend_per_channel.sum()
n_channels = 2
prior_sigma = n_channels * spend_share.to_numpy()

model_config = my_model_config = {
    "intercept": Prior("Normal", mu=0.5, sigma=0.2),
    "saturation_beta": Prior("HalfNormal", sigma=prior_sigma),
    "gamma_control": Prior("Normal", mu=0, sigma=0.05),
    "gamma_fourier": Prior("Laplace", mu=0, b=0.2),
}

config = {
    "date_column": "date_week",
    "channel_columns": ["channel_1", "channel_2"],
    "control_columns": ["price", "event_1", "event_2"],
    "adstock": GeometricAdstock(l_max=4),  # or DelayedAdstock(), etc.
    "saturation": LogisticSaturation(),
    "yearly_seasonality": 2,
    "model_config": model_config,
}

m1 = MMM(**config)

m1.fit(X=X, y=y, chains=4, target_accept=0.85)

In [ ]:
config1 = PyMCConfig(m1, fit_kwargs=fit_kwargs, target_column="quantity")

m2 = MMM(**config1.model_config.config)
m2.fit(X=X, y=y, **config1.fit_config.config)

# Save the config to JSON

In [ ]:
config1.save_config(save_path="tmp/", file_name="config1")

# And load it in a new instantiation

In [ ]:
loaded_config = PyMCConfig().load_config("tmp/config1.json")
loaded_config.model_config.config

# Check if we can run the model from the loaded config

In [ ]:
m3 = MMM(**loaded_config.model_config.config)
m3.fit(X=X, y=y, **loaded_config.fit_config.config)